In [111]:
library(lme4)
library(mbest)
library(arm)
library(dplyr)


Attaching package: ‘dplyr’

The following object is masked from ‘package:MASS’:

    select

The following object is masked from ‘package:nlme’:

    collapse

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [121]:
df = read.csv("soybean_small.csv")

In [122]:
head(df)

,date,plant_stand,precip,temp,hail,crop_hist,area_damaged,severity,seed_tmt,germination,ellip.h,sclerotia,fruit_pods,fruit.spots,seed,mold_growth,seed_discolor,seed_size,shriveling,roots,NA.
1,4,0,2,1,1,1,0,1,0,2,⋯,0,0,4,0,0,0,0,0,0,D1
2,5,0,2,1,0,3,1,1,1,2,⋯,0,0,4,0,0,0,0,0,0,D1
3,3,0,2,1,0,2,0,2,1,1,⋯,0,0,4,0,0,0,0,0,0,D1
4,6,0,2,1,0,1,1,1,0,0,⋯,0,0,4,0,0,0,0,0,0,D1
5,4,0,2,1,0,3,0,2,0,2,⋯,0,0,4,0,0,0,0,0,0,D1
6,5,0,2,1,0,2,0,1,1,0,⋯,0,0,4,0,0,0,0,0,0,D1


In [123]:
str(df)

'data.frame':	47 obs. of  36 variables:
 $ date           : int  4 5 3 6 4 5 3 3 6 6 ...
 $ plant_stand    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ precip         : int  2 2 2 2 2 2 2 2 2 2 ...
 $ temp           : int  1 1 1 1 1 1 1 1 1 1 ...
 $ hail           : int  1 0 0 0 0 0 0 0 0 0 ...
 $ crop_hist      : int  1 3 2 1 3 2 2 1 3 1 ...
 $ area_damaged   : int  0 1 0 1 0 0 1 0 0 0 ...
 $ severity       : int  1 1 2 1 2 1 1 2 1 1 ...
 $ seed_tmt       : int  0 1 1 0 0 1 0 1 1 0 ...
 $ germination    : int  2 2 1 0 2 0 1 2 1 2 ...
 $ plant_growth   : int  1 1 1 1 1 1 1 1 1 1 ...
 $ leaves         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ leafspots_halo : int  0 0 0 0 0 0 0 0 0 0 ...
 $ leafspots_marg : int  2 2 2 2 2 2 2 2 2 2 ...
 $ leafspot_size  : int  2 2 2 2 2 2 2 2 2 2 ...
 $ leaf_shread    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ leaf_malf      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ leaf_mild      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ stem           : int  1 1 1 1 1 1 1 1 1 1 ...
 $ lodging        : int  0 1 

In [125]:
model <- mhglm(fruiting_bodies ~ 1 + precip + (1 + precip | leaves),
                 data = df,
                 family = binomial)

Warning message:
In moment.est(coef, nfixed = rank.fixed, subspace, precision, dispersion, : moment-based covariance matrix estimate is not positive semi-definite; using projection

In [126]:
summary(model)


Call:
mhglm(formula = fruiting_bodies ~ 1 + precip + (1 + precip | 
    leaves), family = binomial, data = df)

Random effects:
 Groups Name        Variance Std.Dev Corr
 leaves (Intercept) 0.012599 0.11225     
        precip      0.049492 0.22247 -1  

Fixed effects:
            Estimate Std. Error z value Pr(>|z|)  
(Intercept)   -3.885      1.911  -2.033   0.0420 *
precip         1.651      1.003   1.647   0.0996 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [127]:
ranef(model)$leaves

,(Intercept),precip
0,0.03589742,-0.07114734
1,-0.03146296,0.06235841


In [105]:
get_priors <- function(model, group, lme4=TRUE) {
  priors <- VarCorr(model)
  if (lme4) {
    priors <- as.data.frame(priors)
    priors <- mutate(priors, var1 = ifelse(var1 == "(Intercept)", "intercept", var1))
    names(priors) <- tolower(names(priors))    
  } else {
    # for now, only take the diagonal
    priors <- data.frame(
        grp   = rep(group, ncol(priors[[group]])),
        var1  = tolower(colnames(priors[[group]])),
        var2  = rep(NA, ncol(priors[[group]])),
        vcov  = diag(priors[[group]]),
        sdcor = sqrt(diag(priors[[group]])),
        # this needs to be here due to bad R quirks
        stringsAsFactors = FALSE
    )
    priors$var1[priors$var1 == '(intercept)'] <- 'intercept'
    rownames(priors) <- NULL
  }
  return(priors)
}

In [106]:
priors = get_priors(model, group="leaves", lme4=FALSE)

In [119]:
priors

,grp,var1,var2,vcov,sdcor
1,leaves,intercept,NA,0.01259922,0.1122462
2,leaves,precip,NA,0.04949194,0.2224678


In [120]:
write.csv(priors, "./soybean_small_priors.csv", row.names=FALSE)